#### **1) Experimenting with Llama 3 model series for entity extraction/generation from given questions.**

- previously llama-2 models have been used for this task. llama-3 models are new series of model and hence experimentation.

- Using llama-3 models (8B and 70B) from [GroqCloud API](https://console.groq.com/playground) since it's fast and currently free for developers.
- combining such prompting technique with Small Language Models?

In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00


In [ ]:
# load question from questions1.json file.
import json
from google.colab import drive

# all files here : https://drive.google.com/drive/folders/148AMfbT_cCmL0gXVADB1kL0cy8cuCG40?usp=drive_link
# all questions present in questions1.json file
## to use the file located in drive :
# 1) create a copy of this google drive in your drive account.
# 2 in colab , in files , mount the google drive.
# 3) select the file-> copy path -> paste it in question_f

question_file_path = "/content/drive/MyDrive/question_answering_files/questions1.json"

def get_questions(question_file_path):
  with open(question_file_path ,'r') as file:
    data = json.load(file)
  return data['questions']



In [ ]:
# iterate over each question to extract key entities present in that question.
from groq import Groq


client = Groq(
    api_key = "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",
)
# list of models : https://console.groq.com/docs/models

def extract_entities(question):
  try:
    chat_completion = client.chat.completions.create(
    messages=[
                {"role": "system", "content": "You are a helpful assistant. Extract key concepts, technical terms, and topics from the user's question. Respond ONLY with a list in this format: [item1, item2, item3]. Do not include any other text or explanations."},
                {"role": "user", "content": question}
            ],
    model="llama-3.1-70b-versatile")
    return chat_completion.choices[0].message.content.strip()
  except Exception as e:
    print(f"Error processing question: {question}")
    print(f"Error: {e}")
    return "Error occurred"


    # prompt1 : "{"role": "system", "content": "You are a helpful assistant. Extract and list the main entities/key topics/technical terms from the given question. strictly Just list entities. nothing else. give answer in list format:[]"},
    # prompt2 : {"role": "system", "content": "You are a helpful assistant. Identify the question and try to extract key concepts/technical terms/topics involved to make it's answering easier. Give answer in List format:[]"}
    # prompt 3 : {"role": "system", "content": "You are a helpful assistant. check the question and try to extract key concepts/technical terms/topics involved to make it's answering easier. format your answer as list only:[]"},
                # {"role": "user", "content": question}
    # prompt4 :{"role": "system", "content": "You are a helpful assistant. Extract key concepts, technical terms, and topics from the user's question. Respond ONLY with a list in this format: [item1, item2, item3]. Do not include any other text or explanations."},


### prompt4 gives the best output amongst all 4.

#### prompt4 uses one-shot prompting providing a single example of the required output.

In [ ]:
from tqdm.notebook import tqdm

# get the questions
questions = get_questions(question_file_path)

# generate the results
results = []
for question in tqdm(questions , desc = "Extracting entities"):
  entities = extract_entities(question)
  results.append({
      "question" : question,
      "key_entities" : entities
  })

Extracting entities:   0%|          | 0/99 [00:00<?, ?it/s]

In [ ]:
# Save results to a new JSON file
output_filename = 'prompt4_entity_result.json'
with open(output_filename, 'w') as f:
    json.dump(results, f, indent=2)

# New Section